In [216]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from urllib.request import urlopen  # for Python 3: from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


######    1- Get the information of FL from Wikipedia 
######    2- FIND THE TABLE, PARSE IT
######    3- ASSIGN DATA TO A DATAFRAME

In [217]:
URL = 'https://en.wikipedia.org/wiki/List_of_municipalities_in_Florida'
soup = BeautifulSoup(urlopen(URL))
htmltable = soup.find("table")
Xrows = soup.find_all(lambda tag: tag.name == "tr" and tag.name.startswith('tr'))
#print(soup.prettify()) # print the original HTML 

rows_Neighbour = []
rows_header = ['Place_name', 'County', 'Population', 'Area', 'Government', 'Label', 'Year']

for Xrow in Xrows:
    rows_Neighbour.append([td.get_text(strip=True) for td in Xrow.find_all(lambda tag: tag.name == "td" and tag.name.startswith('td'))]) 

#print (rows_Neighbour[2:])
dftable = pd.DataFrame(rows_Neighbour[2:], columns=rows_header)

dftable.drop(['Year', 'Label','Government'], axis=1, inplace= True)

# for simpliicty, we will work on the data of centeral FL only 

CentralFL = ['Hillsborough','Pinellas','Orange','Pinellas','Brevard','Polk','Volusia','Osceola','Seminole' ,'Lake']

dfCentralFL = dftable[dftable.set_index(['County']).index.isin(CentralFL)]  

dfCentralFL['Area'] = dfCentralFL['Area'].str.replace(r'\D', '').astype(int) # replace units in the area

dfCentralFL['Population'] = dfCentralFL['Population'].str.replace(r'\D', '').astype(int) # replace units in the Population

dfCentralFL['Place_name'] = dfCentralFL['Place_name'] + ' city'

print(dfCentralFL.shape)

display (dfCentralFL)


(112, 4)


/Users/ramyyoussef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ramyyoussef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ramyyoussef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,Place_name,County,Population,Area
2,Altamonte Springs city,Seminole,44241,952442
6,Apopka city,Orange,53489,2496462
9,Astatula city,Lake,2059,22572
12,Auburndale city,Polk,16291,932412
17,Bartow city,Polk,19926,52313562
20,Bay Lake city,Orange,51,2115472
23,Belle Isle city,Orange,7166,461202
24,Belleair city,Pinellas,4217,28732
25,Belleair Beach city,Pinellas,1609,18452
26,Belleair Bluffs city,Pinellas,2199,7162


    Add Co-ordinates to our dataframe  

In [218]:
coordinatesURL = 'https://www.mapsofworld.com/usa/states/florida/lat-long.html'
coordinatessoup = BeautifulSoup(urlopen(coordinatesURL))
coordinateshtmltable = coordinatessoup.find("table")
coordinatesXrows = coordinatessoup.find_all(lambda tag: tag.name == "tr" and tag.name.startswith('tr'))

rows_coordinates = []


for coordinatesXrow in coordinatesXrows:
    rows_coordinates.append([td.get_text(strip=True) for td in coordinatesXrow.find_all(lambda tag: tag.name == "td" and tag.name.startswith('td'))]) 

coordinatesdftable = pd.DataFrame(rows_coordinates[4:925], columns=['Place_name','Latitude','longitude'])

#display (coordinatesdftable)
dftable_merged = pd.merge(dfCentralFL, coordinatesdftable, on='Place_name')
dftable_merged



,Place_name,County,Population,Area,Latitude,longitude
0,Altamonte Springs city,Seminole,44241,952442,28.66,-81.39
1,Apopka city,Orange,53489,2496462,28.7,-81.53
2,Auburndale city,Polk,16291,932412,28.1,-81.8
3,Bartow city,Polk,19926,52313562,27.89,-81.82
4,Bay Lake city,Orange,51,2115472,28.39,-81.58
5,Belle Isle city,Orange,7166,461202,28.47,-81.35
6,Belleair Beach city,Pinellas,1609,18452,27.92,-82.84
7,Belleair Bluffs city,Pinellas,2199,7162,27.92,-82.82
8,Cape Canaveral city,Brevard,10449,23602,28.39,-80.61
9,Casselberry city,Seminole,28876,711842,28.66,-81.32


In [121]:
CLIENT_ID = 'VETD5IR5PXCKYOQQTZN34BP2ZH0OGVXGU0KHN3OG0B2INCWB' # your Foursquare ID
CLIENT_SECRET = '1LOS5415UTZ4FTO4WEJ1JJ44SZ2YNYHOYJUDPIMZCKZN3KS2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



neighborhood_latitude = dftable_merged.loc[8, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dftable_merged.loc[8, 'longitude'] # neighborhood longitude value

neighborhood_name = dftable_merged.loc[8, 'Place_name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

search_query = 'Altamonte Springs'
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: VETD5IR5PXCKYOQQTZN34BP2ZH0OGVXGU0KHN3OG0B2INCWB
CLIENT_SECRET:1LOS5415UTZ4FTO4WEJ1JJ44SZ2YNYHOYJUDPIMZCKZN3KS2
Latitude and longitude values of Cape Canaveral city are 28.39, -80.61.


{'meta': {'code': 200, 'requestId': '5e8bb2ea47e0d60026130156'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cape Canaveral',
  'headerFullLocation': 'Cape Canaveral',
  'headerLocationGranularity': 'city',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 28.394500004500006,
    'lng': -80.60489434888785},
   'sw': {'lat': 28.385499995499995, 'lng': -80.61510565111215}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58fb66e5a879217add28b9f8',
       'name': 'Southern Charm Cafe',
       'location': {'address': '8501 A1a',
        'lat': 28.39296787177543,
        'lng': -80.60966433924438,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.3929678717

In [219]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [220]:

from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/ramyyoussef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Southern Charm Cafe,Breakfast Spot,28.392968,-80.609664
1,Twistee Treat Cape Canaveral,Ice Cream Shop,28.391258,-80.607345
2,Cumberland Farms,Convenience Store,28.388819,-80.605871
3,Budget Car Rental,Rental Car Location,28.392130,-80.609122
4,95 South,Moving Target,28.387648,-80.612367


In [221]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [125]:
CentralFL_venues = getNearbyVenues(names=dftable_merged['Place_name'],
                                   latitudes=dftable_merged['Latitude'],
                                   longitudes=dftable_merged['longitude']
                                  )

Altamonte Springs city
Apopka city
Auburndale city
Bartow city
Bay Lake city
Belle Isle city
Belleair Beach city
Belleair Bluffs city
Cape Canaveral city
Casselberry city
Clearwater city
Clermont city
Cocoa city
Cocoa Beach city
Davenport city
Daytona Beach city
Daytona Beach Shores city
DeBary city
Deltona city
Dunedin city
Eagle Lake city
Edgewater city
Edgewood city
Eustis city
Fort Meade city
Frostproof city
Fruitland Park city
Groveland city
Gulfport city
Haines City city
Holly Hill city
Indian Harbour Beach city
Indian Rocks Beach city
Kissimmee city
Lake Alfred city
Lake Buena Vista city
Lake Helen city
Lake Mary city
Lake Wales city
Lakeland city
Largo city
Leesburg city
Longwood city
Madeira Beach city
Maitland city
Mascotte city
Melbourne city
Minneola city
Mount Dora city
Mulberry city
New Smyrna Beach city
Oak Hill city
Ocoee city
Oldsmar city
Orange City city
Orlando city
Ormond Beach city
Oviedo city
Palm Bay city
Pinellas Park city
Plant City city
Port Orange city
Rockle

In [222]:
print(CentralFL_venues.shape)
CentralFL_venues.head()

(354, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altamonte Springs city,28.66,-81.39,Christmas Tree Shops,28.660224,-81.386824,Gift Shop
1,Altamonte Springs city,28.66,-81.39,Altamonte Springs Yoga,28.662508,-81.392422,Yoga Studio
2,Altamonte Springs city,28.66,-81.39,Chick-fil-A,28.661289,-81.392580,Fast Food Restaurant
3,Altamonte Springs city,28.66,-81.39,LongHorn Steakhouse,28.662972,-81.389751,Steakhouse
4,Altamonte Springs city,28.66,-81.39,Twin Peaks,28.662467,-81.392541,Sports Bar


In [223]:
CentralFL_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Altamonte Springs city,25,25,25,25,25,25
Apopka city,4,4,4,4,4,4
Bartow city,1,1,1,1,1,1
Belleair Beach city,5,5,5,5,5,5
Belleair Bluffs city,11,11,11,11,11,11
Cape Canaveral city,11,11,11,11,11,11
Casselberry city,1,1,1,1,1,1
Clearwater city,5,5,5,5,5,5
Clermont city,19,19,19,19,19,19


In [224]:
print('There are {} uniques categories.'.format(len(CentralFL_venues['Venue Category'].unique())))

There are 131 uniques categories.


In [225]:
# one hot encoding
CentralFL_onehot = pd.get_dummies(CentralFL_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CentralFL_onehot['Neighborhood'] = CentralFL_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CentralFL_onehot.columns[-1]] + list(CentralFL_onehot.columns[:-1])
CentralFL_onehot = CentralFL_onehot[fixed_columns]

CentralFL_onehot.head()


,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Bike Rental / Bike Share,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Donut Shop,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gift Shop,Golf Course,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Liquor Store,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Neighborhood,New American Restaurant,Office,Optical Shop,Paintball Field,Park,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racetrack,Recreation Center,Rental Car Location,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Smoke Shop,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Taco Place,Tea Room,Tennis Court,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Volleyball Court,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Altamonte Springs city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Altamonte Springs city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Altamonte Springs city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Altamonte Springs city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Altamonte Springs city,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [226]:
CentralFL_onehot.shape

(354, 131)

In [227]:
df = CentralFL_onehot
Neighborhood = df['Neighborhood']
df.drop(labels=['Neighborhood'], axis=1,inplace = True)
df.insert(0, 'Neighborhood', Neighborhood)

df_CentralFL_grouped = df.groupby('Neighborhood').mean().reset_index()
df_ClusterInput = df_CentralFL_grouped.T
display(df_ClusterInput)






# one hot encoding
CentralFL_onehot = pd.get_dummies(CentralFL_venues[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CentralFL_onehot['Venue Category'] = CentralFL_venues['Venue Category'] 

# move neighborhood column to the first column
fixed_columns = [CentralFL_onehot.columns[-1]] + list(CentralFL_onehot.columns[:-1])
CentralFL_onehot = CentralFL_onehot[fixed_columns]

CentralFL_onehot.head()


,Venue Category,Altamonte Springs city,Apopka city,Bartow city,Belleair Beach city,Belleair Bluffs city,Cape Canaveral city,Casselberry city,Clearwater city,Clermont city,Cocoa city,Davenport city,Daytona Beach Shores city,Daytona Beach city,DeBary city,Deltona city,Eagle Lake city,Edgewood city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city,Indian Harbour Beach city,Kissimmee city,Lake Alfred city,Lake Buena Vista city,Lake Helen city,Lake Mary city,Lakeland city,Largo city,Longwood city,Maitland city,Melbourne city,Minneola city,Mount Dora city,New Smyrna Beach city,Ocoee city,Oldsmar city,Orange City city,Orlando city,Ormond Beach city,Oviedo city,Palm Bay city,Pinellas Park city,Plant City city,Rockledge city,Safety Harbor city,Sanford city,Seminole city,South Daytona city,South Pasadena city,St. Petersburg city,Tampa city,Tarpon Springs city,Tavares city,Temple Terrace city,Titusville city,Treasure Island city,Umatilla city,West Melbourne city,Winter Garden city,Winter Haven city,Winter Park city
0,Gift Shop,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Yoga Studio,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fast Food Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Steakhouse,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Sports Bar,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [253]:
CentralFL_grouped = CentralFL_onehot.groupby('Venue Category').mean().reset_index()
CentralFL_grouped

,Venue Category,Altamonte Springs city,Apopka city,Bartow city,Belleair Beach city,Belleair Bluffs city,Cape Canaveral city,Casselberry city,Clearwater city,Clermont city,Cocoa city,Davenport city,Daytona Beach Shores city,Daytona Beach city,DeBary city,Deltona city,Eagle Lake city,Edgewood city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city,Indian Harbour Beach city,Kissimmee city,Lake Alfred city,Lake Buena Vista city,Lake Helen city,Lake Mary city,Lakeland city,Largo city,Longwood city,Maitland city,Melbourne city,Minneola city,Mount Dora city,New Smyrna Beach city,Ocoee city,Oldsmar city,Orange City city,Orlando city,Ormond Beach city,Oviedo city,Palm Bay city,Pinellas Park city,Plant City city,Rockledge city,Safety Harbor city,Sanford city,Seminole city,South Daytona city,South Pasadena city,St. Petersburg city,Tampa city,Tarpon Springs city,Tavares city,Temple Terrace city,Titusville city,Treasure Island city,Umatilla city,West Melbourne city,Winter Garden city,Winter Haven city,Winter Park city
0,American Restaurant,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.500000,0.000000,0.0,0.000000,0.000000
1,Aquarium,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,Art Gallery,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Asian Restaurant,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,Automotive Shop,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,BBQ Joint,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.00000

In [229]:
def return_most_crowded_places(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [254]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue Category']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Crowded City with this Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Crowded City with this Venue'.format(ind+1))

# create a new dataframe
venues_neighborhoods_sorted = pd.DataFrame(columns=columns)
venues_neighborhoods_sorted['Venue Category'] = CentralFL_grouped['Venue Category']

for ind in np.arange(CentralFL_grouped.shape[0]):
    venues_neighborhoods_sorted.iloc[ind, 1:] = return_most_common_venues(CentralFL_grouped.iloc[ind, :], num_top_venues)

venues_neighborhoods_sorted.head()

,Venue Category,1st Crowded City with this Venue,2nd Crowded City with this Venue,3rd Crowded City with this Venue,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
0,American Restaurant,Umatilla city,Altamonte Springs city,Treasure Island city,Holly Hill city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Kissimmee city
1,Aquarium,Longwood city,Winter Haven city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
2,Art Gallery,Belleair Beach city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
3,Asian Restaurant,Clermont city,Seminole city,Indian Harbour Beach city,Winter Park city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city
4,Automotive Shop,Longwood city,Altamonte Springs city,Winter Garden city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city


In [255]:
# set number of clusters
kclusters = 5

CentralFL_grouped_clustering = CentralFL_grouped.drop('Venue Category', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CentralFL_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 0, 3, 1, 3, 3, 3], dtype=int32)

In [256]:
# add clustering labels
venues_neighborhoods_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CentralFL_merged = dftable_merged


In [257]:
venues_neighborhoods_sorted.loc[venues_neighborhoods_sorted['Cluster Labels'] == 0, 
                     venues_neighborhoods_sorted.columns[[1] + list(range(5, venues_neighborhoods_sorted.shape[1]))]]

,Venue Category,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
1,Aquarium,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
4,Automotive Shop,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
57,Health Food Store,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Indian Harbour Beach city,Holly Hill city
60,Hobby Shop,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
96,Platform,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
100,Pub,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
117,Taco Place,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
123,Train Station,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city


In [260]:
venues_neighborhoods_sorted.loc[venues_neighborhoods_sorted['Cluster Labels'] == 1]

,Cluster Labels,Venue Category,1st Crowded City with this Venue,2nd Crowded City with this Venue,3rd Crowded City with this Venue,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
6,1,Baby Store,Altamonte Springs city,Winter Haven city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
22,1,Clothing Store,Clermont city,Altamonte Springs city,Treasure Island city,Holly Hill city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Indian Harbour Beach city
27,1,Cosmetics Shop,Clermont city,Kissimmee city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Indian Harbour Beach city
47,1,Gift Shop,Altamonte Springs city,Winter Haven city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
67,1,Indian Restaurant,Altamonte Springs city,Winter Haven city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
68,1,Indie Movie Theater,Clermont city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
72,1,Japanese Restaurant,Clermont city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
80,1,Middle Eastern Restaurant,Clermont city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
85,1,Movie Theater,Altamonte Springs city,Winter Haven city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
87,1,Multiplex,Clermont city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city


In [259]:
venues_neighborhoods_sorted.loc[venues_neighborhoods_sorted['Cluster Labels'] == 2]

,Cluster Labels,Venue Category,1st Crowded City with this Venue,2nd Crowded City with this Venue,3rd Crowded City with this Venue,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
38,2,Event Space,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
55,2,Hardware Store,Treasure Island city,Clermont city,Winter Park city,Holly Hill city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Indian Harbour Beach city
62,2,Hot Dog Joint,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
63,2,Hotel,Treasure Island city,Altamonte Springs city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
74,2,Karaoke Bar,Lakeland city,Treasure Island city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
83,2,Motel,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
88,2,Neighborhood,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
89,2,New American Restaurant,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
104,2,Resort,Treasure Island city,Lake Buena Vista city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
106,2,Salon / Barbershop,Treasure Island city,Winter Park city,Eagle Lake city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city


In [261]:
venues_neighborhoods_sorted.loc[venues_neighborhoods_sorted['Cluster Labels'] == 3]

,Cluster Labels,Venue Category,1st Crowded City with this Venue,2nd Crowded City with this Venue,3rd Crowded City with this Venue,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
0,3,American Restaurant,Umatilla city,Altamonte Springs city,Treasure Island city,Holly Hill city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Kissimmee city
2,3,Art Gallery,Belleair Beach city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
3,3,Asian Restaurant,Clermont city,Seminole city,Indian Harbour Beach city,Winter Park city,Fort Meade city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city
5,3,BBQ Joint,Tampa city,Fort Meade city,Kissimmee city,Titusville city,Plant City city,Winter Park city,Edgewood city,Frostproof city,Fruitland Park city,Gulfport city
7,3,Bakery,Belleair Bluffs city,Fruitland Park city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city
8,3,Bar,Treasure Island city,Longwood city,Ormond Beach city,Apopka city,Clearwater city,Clermont city,Indian Harbour Beach city,Largo city,Altamonte Springs city,West Melbourne city
9,3,Baseball Field,Orange City city,Sanford city,Daytona Beach Shores city,Winter Park city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city,Edgewood city
10,3,Baseball Stadium,Mount Dora city,Winter Park city,Largo city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city,Holly Hill city
11,3,Basketball Court,Bartow city,Kissimmee city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Indian Harbour Beach city
12,3,Beach,Treasure Island city,Tarpon Springs city,Belleair Beach city,Cape Canaveral city,Indian Harbour Beach city,Frostproof city,Fruitland Park city,Gulfport city,Haines City city,Holly Hill city


In [262]:
venues_neighborhoods_sorted.loc[venues_neighborhoods_sorted['Cluster Labels'] == 4]

,Cluster Labels,Venue Category,1st Crowded City with this Venue,2nd Crowded City with this Venue,3rd Crowded City with this Venue,4th Crowded City with this Venue,5th Crowded City with this Venue,6th Crowded City with this Venue,7th Crowded City with this Venue,8th Crowded City with this Venue,9th Crowded City with this Venue,10th Crowded City with this Venue
28,4,Cuban Restaurant,Cape Canaveral city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
86,4,Moving Target,Cape Canaveral city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
103,4,Rental Car Location,Cape Canaveral city,Winter Park city,Edgewood city,Lakeland city,Lake Mary city,Lake Helen city,Lake Buena Vista city,Lake Alfred city,Kissimmee city,Indian Harbour Beach city
